## Kubernetes Clusterwide Kafka mTLS Demo

### Setup

Create a Kind cluster by using an ansible playbook from the project `ansible` folder.


In [ ]:
!cd ../ansible && ansible-playbook playbooks/kind-cluster.yaml

In [ ]:
!cd ../ansible && ansible-playbook playbooks/setup-ecosystem.yaml

In [172]:
!helm upgrade --install seldon-core-v2-crds  ../k8s/helm-charts/seldon-core-v2-crds -n seldon-mesh

Release "seldon-core-v2-crds" has been upgraded. Happy Helming!
NAME: seldon-core-v2-crds
LAST DEPLOYED: Thu Jun 22 15:22:04 2023
NAMESPACE: seldon-mesh
STATUS: deployed
REVISION: 5
TEST SUITE: None


In [173]:
!kubectl create namespace ns1

namespace/ns1 created


In [174]:
!kubectl create namespace ns2

namespace/ns2 created


In [175]:
!cat ../k8s/samples/values-strimzi-kafka-mtls.yaml

---
kafka:
  bootstrap: seldon-kafka-bootstrap.seldon-mesh.svc.cluster.local:9093

security:
  kafka:
    protocol: SSL
    ssl:
      client:
        secret: seldon
        brokerValidationSecret: seldon-cluster-ca-cert
        keyPath: /tmp/certs/kafka/client/user.key
        crtPath: /tmp/certs/kafka/client/user.crt
        caPath: /tmp/certs/kafka/client/ca.crt
        brokerCaPath: /tmp/certs/kafka/broker/ca.crt


In [176]:
!helm install seldon-v2 ../k8s/helm-charts/seldon-core-v2-setup/ -n seldon-mesh --set controller.clusterwide=true --values ../k8s/samples/values-strimzi-kafka-mtls.yaml

NAME: seldon-v2
LAST DEPLOYED: Thu Jun 22 15:22:11 2023
NAMESPACE: seldon-mesh
STATUS: deployed
REVISION: 1
TEST SUITE: None


## Copy Strimzi secrets to namespaces

Strimzi doesn't allow a single Kafka cluster to be used in TLS easily from multiple namespaces without copying the user Secrets created by the KafkaUser to those namespaces.

In [177]:
!kubectl get secret seldon -n seldon-mesh -o json | jq 'del(.metadata.ownerReferences) | del(.metadata.namespace)' | kubectl create -f - -n ns1

secret/seldon created


In [178]:
!kubectl get secret seldon-cluster-ca-cert -n seldon-mesh -o json | jq 'del(.metadata.ownerReferences) | del(.metadata.namespace)' | kubectl create -f - -n ns1

secret/seldon-cluster-ca-cert created


In [179]:
!kubectl get secret seldon -n seldon-mesh -o json | jq 'del(.metadata.ownerReferences) | del(.metadata.namespace)' | kubectl create -f - -n ns2

secret/seldon created


In [180]:
!kubectl get secret seldon-cluster-ca-cert -n seldon-mesh -o json | jq 'del(.metadata.ownerReferences) | del(.metadata.namespace)' | kubectl create -f - -n ns2

secret/seldon-cluster-ca-cert created


## Create SeldonRuntimes

In [181]:
!helm install seldon-v2-runtime ../k8s/helm-charts/seldon-core-v2-runtime  -n ns1 --wait

NAME: seldon-v2-runtime
LAST DEPLOYED: Thu Jun 22 15:22:19 2023
NAMESPACE: ns1
STATUS: deployed
REVISION: 1
TEST SUITE: None


In [182]:
!helm install seldon-v2-runtime ../k8s/helm-charts/seldon-core-v2-runtime  -n ns2 --wait

NAME: seldon-v2-runtime
LAST DEPLOYED: Thu Jun 22 15:22:20 2023
NAMESPACE: ns2
STATUS: deployed
REVISION: 1
TEST SUITE: None


In [183]:
!helm install seldon-v2-servers ../k8s/helm-charts/seldon-core-v2-servers  -n ns1 --wait

NAME: seldon-v2-servers
LAST DEPLOYED: Thu Jun 22 15:22:23 2023
NAMESPACE: ns1
STATUS: deployed
REVISION: 1
TEST SUITE: None


In [184]:
!helm install seldon-v2-servers ../k8s/helm-charts/seldon-core-v2-servers  -n ns2 --wait

NAME: seldon-v2-servers
LAST DEPLOYED: Thu Jun 22 15:22:24 2023
NAMESPACE: ns2
STATUS: deployed
REVISION: 1
TEST SUITE: None


In [185]:
!kubectl wait --for condition=ready --timeout=300s server --all -n ns1

server.mlops.seldon.io/mlserver condition met
server.mlops.seldon.io/triton condition met


In [186]:
!kubectl wait --for condition=ready --timeout=300s server --all -n ns2

server.mlops.seldon.io/mlserver condition met
server.mlops.seldon.io/triton condition met


### Get Inference Endpoints

In [187]:
MESH_IP=!kubectl get svc seldon-mesh -n ns1 -o jsonpath='{.status.loadBalancer.ingress[0].ip}'
MESH_IP_NS1=MESH_IP[0]
import os
os.environ['MESH_IP_NS1'] = MESH_IP_NS1
MESH_IP_NS1

'172.21.255.2'

In [188]:
MESH_IP=!kubectl get svc seldon-mesh -n ns2 -o jsonpath='{.status.loadBalancer.ingress[0].ip}'
MESH_IP_NS2=MESH_IP[0]
import os
os.environ['MESH_IP_NS2'] = MESH_IP_NS2
MESH_IP_NS2

'172.21.255.4'

### Launch Pipeline in Namespace ns1

In [189]:
!kubectl create -f ./models/tfsimple1.yaml -n ns1
!kubectl create -f ./models/tfsimple2.yaml -n ns1

model.mlops.seldon.io/tfsimple1 created
model.mlops.seldon.io/tfsimple2 created


In [190]:
!kubectl wait --for condition=ready --timeout=300s model --all -n ns1

model.mlops.seldon.io/tfsimple1 condition met
model.mlops.seldon.io/tfsimple2 condition met


In [191]:
!kubectl create -f ./pipelines/tfsimples.yaml -n ns1

pipeline.mlops.seldon.io/tfsimples created


In [192]:
!kubectl wait --for condition=ready --timeout=300s pipeline --all -n ns1

pipeline.mlops.seldon.io/tfsimples condition met


In [193]:
!seldon pipeline infer tfsimples --inference-mode grpc --inference-host ${MESH_IP_NS1}:80 \
    '{"model_name":"simple","inputs":[{"name":"INPUT0","contents":{"int_contents":[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]},"datatype":"INT32","shape":[1,16]},{"name":"INPUT1","contents":{"int_contents":[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]},"datatype":"INT32","shape":[1,16]}]}' | jq -M .

{
  "outputs": [
    {
      "name": "OUTPUT0",
      "datatype": "INT32",
      "shape": [
        "1",
        "16"
      ],
      "contents": {
        "intContents": [
          2,
          4,
          6,
          8,
          10,
          12,
          14,
          16,
          18,
          20,
          22,
          24,
          26,
          28,
          30,
          32
        ]
      }
    },
    {
      "name": "OUTPUT1",
      "datatype": "INT32",
      "shape": [
        "1",
        "16"
      ],
      "contents": {
        "intContents": [
          2,
          4,
          6,
          8,
          10,
          12,
          14,
          16,
          18,
          20,
          22,
          24,
          26,
          28,
          30,
          32
        ]
      }
    }
  ]
}


### Launch Pipeline in Namespace ns2

In [194]:
!kubectl create -f ./models/tfsimple1.yaml -n ns2
!kubectl create -f ./models/tfsimple2.yaml -n ns2

model.mlops.seldon.io/tfsimple1 created
model.mlops.seldon.io/tfsimple2 created


In [195]:
!kubectl wait --for condition=ready --timeout=300s model --all -n ns2

model.mlops.seldon.io/tfsimple1 condition met
model.mlops.seldon.io/tfsimple2 condition met


In [196]:
!kubectl create -f ./pipelines/tfsimples.yaml -n ns2

pipeline.mlops.seldon.io/tfsimples created


In [197]:
!kubectl wait --for condition=ready --timeout=300s pipeline --all -n ns2

pipeline.mlops.seldon.io/tfsimples condition met


In [199]:
!seldon pipeline infer tfsimples --inference-mode grpc --inference-host ${MESH_IP_NS2}:80 \
    '{"model_name":"simple","inputs":[{"name":"INPUT0","contents":{"int_contents":[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]},"datatype":"INT32","shape":[1,16]},{"name":"INPUT1","contents":{"int_contents":[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]},"datatype":"INT32","shape":[1,16]}]}' | jq -M .

{
  "outputs": [
    {
      "name": "OUTPUT0",
      "datatype": "INT32",
      "shape": [
        "1",
        "16"
      ],
      "contents": {
        "intContents": [
          2,
          4,
          6,
          8,
          10,
          12,
          14,
          16,
          18,
          20,
          22,
          24,
          26,
          28,
          30,
          32
        ]
      }
    },
    {
      "name": "OUTPUT1",
      "datatype": "INT32",
      "shape": [
        "1",
        "16"
      ],
      "contents": {
        "intContents": [
          2,
          4,
          6,
          8,
          10,
          12,
          14,
          16,
          18,
          20,
          22,
          24,
          26,
          28,
          30,
          32
        ]
      }
    }
  ]
}


## TearDown

In [200]:
!kubectl delete -f ./pipelines/tfsimples.yaml -n ns1
!kubectl delete -f ./pipelines/tfsimples.yaml -n ns2
!kubectl delete -f ./models/tfsimple1.yaml -n ns1
!kubectl delete -f ./models/tfsimple2.yaml -n ns1
!kubectl delete -f ./models/tfsimple1.yaml -n ns2
!kubectl delete -f ./models/tfsimple2.yaml -n ns2

pipeline.mlops.seldon.io "tfsimples" deleted
pipeline.mlops.seldon.io "tfsimples" deleted
model.mlops.seldon.io "tfsimple1" deleted
model.mlops.seldon.io "tfsimple2" deleted
model.mlops.seldon.io "tfsimple1" deleted
model.mlops.seldon.io "tfsimple2" deleted


In [201]:
!helm delete seldon-v2-servers -n ns1 --wait

release "seldon-v2-servers" uninstalled


In [202]:
!helm delete seldon-v2-runtime -n ns1 --wait

release "seldon-v2-runtime" uninstalled


In [203]:
!helm delete seldon-v2-servers -n ns2 --wait

release "seldon-v2-servers" uninstalled


In [204]:
!helm delete seldon-v2-runtime -n ns2 --wait

release "seldon-v2-runtime" uninstalled


In [205]:
!helm delete seldon-v2  -n seldon-mesh

release "seldon-v2" uninstalled


In [206]:
!kubectl delete namespace ns1
!kubectl delete namespace ns2

namespace "ns1" deleted
namespace "ns2" deleted
